<a href="https://colab.research.google.com/github/wahyunh10/AB-testing-Ecommerce/blob/main/ab_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analyze A/B Test Results**

# **Table of Contents**
* Introduction
* Part I - Probability
* Part II - A/B Test
* Part III - Regression

# **Introduction**

A/B tests are very commonly performed by data analysts and data scientists. It is important that you get some practice working with the difficulties of these

For this project, you will be working to understand the results of an A/B test run by an e-commerce website. Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

# **Part I - Probability**
To get started, let's import our libraries.

In [2]:
import pandas as pd
import numpy as np
import random 
import time
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

a. Read in the dataset and take a look at the top few rows here:

In [3]:
#reading the dataset
df = pd.read_csv('ab_testing_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the cell below to find the number of rows in the dataset.

In [4]:
#Number of rows
df.shape[0]

294478

c. The number of unique users in the dataset.

In [5]:
#Number of unique users
df['user_id'].nunique()

290584

d. The proportion of users converted.

In [6]:
#Proportion of users converted
len(df[df['converted'] ==1]) / df['user_id'].nunique()

0.12126269856564711

e. The number of times the new_page and treatment don't match.

In [7]:
#Number of rows that follows:
#When group is treatment but landing_page is not new_page,
#When group is not treatment but landing_page is new_page
df2 = df.query('(group == "treatment" & landing_page != "new_page") or (group != "treatment" & landing_page == "new_page")')
df2.shape[0]

3893

f. Do any of the rows have missing values?

In [8]:
#Checking data for Null Values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz. Store your new dataframe in df2.

In [9]:
#Dropping rows that may have wrong information
df2 = df.drop(df.query('(group == "treatment" & landing_page != "new_page") or (group != "treatment" & landing_page == "new_page")').index)

In [10]:
#Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

a. How many **unique user_ids** are in **df2**?

In [11]:
#Unique users in the new dataframe
df2.user_id.nunique()

290584

b. There is **one user_id** repeated in **df2**. What is it?

In [12]:
#Duplicated user
df2[df2.duplicated(['user_id'])]['user_id']

2893    773192
Name: user_id, dtype: int64

c. What is the row information for the **repeat user_id**?

In [13]:
#Row of the duplicated user_id
df2[df2['user_id'].duplicated() == True]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


d. **Remove one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [14]:
#Dropping duplicated user_id
df2 = df2.drop(index=2893)

a. What is the probability of an individual converting regardless of the page they receive?

In [15]:
#Probability of converting regardless of the page
len(df2[df2['converted'] ==1]) / df2['user_id'].nunique()

0.11959708724499628

b. Given that an individual was in the control group, what is the probability they converted?

In [16]:
#Probability of converting when group is control
len(df2.query('group == "control" & converted == 1')) / len(df2.query('group == "control"'))

0.1203863045004612

c. Given that an individual was in the treatment group, what is the probability they converted?

In [17]:
#Probability of converting when group is treatment
len(df2.query('group == "treatment" & converted == 1')) / len(df2.query('group == "treatment"'))

0.11880806551510564

d. What is the probability that an individual received the new page?

In [18]:
#Probability of landing on the new page
len(df2.query('landing_page == "new_page"')) / df2.shape[0]

0.5000619442226688

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

**There is no sufficient evidence prove that new treatment page leads more conversions as questions 'B' & 'C' clarify that Old control page has slightly higher conversion rate.**

# **Part II - A/B Test**

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time? How long do you run to render a decision that neither page is better than another?

These questions are the difficult parts associated with A/B tests in general.

1. For now, consider you need to make the decision just based on all the data provided. If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be? You can state your hypothesis in terms of words or in terms of  and , which are the converted rates for the old and new pages.

**Null Hypotheses:**

Old page is better than or equal new page

**P_𝑜𝑙𝑑 >= P_𝑛𝑒𝑤**


**Alternative Hypotheses:**

Old page is worse than new page

**P_𝑜𝑙𝑑 < P_𝑛𝑒𝑤**

2. Assume under the null hypothesis, P_𝑛𝑒𝑤 and P_𝑜𝑙𝑑  both have "true" success rates equal to the converted success rate regardless of page - that is P_𝑛𝑒𝑤  and P_𝑜𝑙𝑑  are equal. Furthermore, assume they are equal to the converted rate in ab_testing_data.csv regardless of the page.

Use a sample size for each page equal to the ones in ab_testing_data.csv.

Perform the sampling distribution for the difference in converted between the two pages over 10,000 iterations of calculating an estimate from the null.

Use the cells below to provide the necessary parts of this simulation. If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem. You can use Quiz 5 in the classroom to make sure you are on the right track.

a. What is the **conversion rate for P_𝑛𝑒𝑤** under the null?

In [19]:
#Conversuin of P_new rate under null
p_new = len(df2.query('converted == 1 '))/df['user_id'].nunique()
p_new

0.11959708724499628

b. What is the **conversion rate for P_𝑜𝑙𝑑** under the null?

In [20]:
#Conversuin of P_old rate under null
p_old = len(df2.query('converted == 1 '))/df['user_id'].nunique()
p_old

0.11959708724499628

c. What is *n_new* the number of individuals in the treatment group?

In [21]:
#Number of individulas in the treatment group
n_new = df2.query('group == "treatment" &landing_page == "new_page"')['user_id'].nunique()
n_new

145310

d. What is *n_new*  the number of individuals in the control group?

In [22]:
#Number of individuals in the control group
n_old = df2.query('group == "control" & landing_page == "old_page"')['user_id'].nunique()
n_old

145274

e. Simulate *n_new* transactions with a conversion rate of  P_𝑛𝑒𝑤 under the null. Store these  1's and 0's in **new_page_converted**.

In [23]:
#Simulating the conversion rate of the new page uner null
new_page_converted = np.random.choice([0,1],size= n_new,p=(p_new,1-p_new))
new_page_converted.mean()

0.8812332255178583